In [3]:
library(rstan)

In [4]:
# The Stan model

num_data <- 339
n <- 270
n_future <- num_data - n

data_full <- read.table("CS146_data_reduced.csv", header=FALSE, sep=",")
x <- data_full$V1
y <- data_full$V2

data <- list(
  n = n,
  n_future = n_future,
  num_data = num_data,
  x = x[1:num_data],
  y = y[1:n])

model <- "
data {
  int<lower=0> n;
  int<lower=0> n_future;
  int<lower=0> num_data;
  real x[num_data];
  real y[n];
}

parameters {
  real c_0;
  real c_1;
  real c_2;
  real c_6_prime;
  real c_7_prime1;
  real c_7_prime2;
  real c_8_prime;
}

transformed parameters {
  real<lower=0> c_6;
  real<lower=-pi(),upper=pi()> c_7;
  real<lower=0> c_8;

  c_6 = exp(c_6_prime);
  c_7 = atan2(c_7_prime1, c_7_prime2);
  c_8 = exp(c_8_prime);

}

model {
  c_0 ~ normal(0, 1);
  c_1 ~ normal(0, 1);
  c_2 ~ normal(0, 1);
  c_6_prime ~ normal(-4,1);
  c_7_prime1 ~ normal(0, 1);
  c_7_prime2 ~ normal(0, 1);
  c_8_prime ~ normal(-4.4,3);
  for(t in 1:n) {
    y[t] ~ normal(c_0*x[t]*x[t] + c_1*x[t] + c_2 - c_6*(sin((2*pi()/365.25)*21791*x[t]-
            c_7)+0.25*sin(2*(2*pi()/365.25)*21791*x[t]-2*c_7)), c_8);
  }
}

generated quantities {
  real y_future[n_future];
  for(t in 1:n_future) {
    y_future[t] = normal_rng(c_0*x[t+n]*x[t+n] + c_1*x[t+n] + c_2 - c_6*(sin((2*pi()
            /365.25)*21791*x[t+n]-c_7)+0.25*sin(2*(2*pi()/365.25)*21791*x[t+n]-2*c_7))
            , c_8);
  }
}
"

In [9]:
# Fit the Stan model.
fit <- stan(
  model_code = model,
  data = data,
  chains = 4,
  warmup = 2000,
  iter = 6000,
  cores = 3,
  refresh = 1000,
  control = list(adapt_delta = 0.999)
)

print(fit, par=c('c_0', 'c_1', 'c_2', 'c_6', 'c_7', 'c_8'), probs=c(.05, .5, 0.95))
samples <- extract(fit)

In [10]:
# Plot confidence intervals
results <- apply(samples$y_future, 2, quantile, probs=c(0.025, 0.975))

plot(
  x[1:(n+n_future)], y[1:(n+n_future)],
  col='black', type='l',
  xlim=c(x[1], x[n+n_future]),
  ylim=c(min(c(results[1,], y)), max(c(results[2,], y))),
  main='Data, future data, and predicted 95% interval')
lines(x[n:(n+n_future)], c(y[n], results[1,]), lty='dashed', col='blue')
lines(x[n:(n+n_future)], c(y[n], results[2,]), lty='dashed', col='blue')
abline(v=n, col='red')

In [11]:
# Plot of sample autocorrelation

acf(samples$c_0, main="Autocorrelation of c_0 samples")
acf(samples$c_1, main="Autocorrelation of c_1 samples")
acf(samples$c_2, main="Autocorrelation of c_2 samples")
acf(samples$c_6, main="Autocorrelation of c_6 samples")
acf(samples$c_7, main="Autocorrelation of c_7 samples")
acf(samples$c_8, main="Autocorrelation of c_8 samples")

In [4]:
library(rstan)

In [6]:
# The Stan model

num_data <- 339
n <- 270
n_future <- 227

data_full <- read.table("CS146_data_reduced.csv", header=FALSE, sep=",")
x <- data_full$V1
y <- data_full$V2

data <- list(
  n = n,
  n_future = n_future,
  num_data = num_data,
  x = x[1:num_data+n_future],
  y = y[1:num_data])

model <- "
data {
  int<lower=0> n;
  int<lower=0> n_future;
  int<lower=0> num_data;
  real x[num_data+n_future];
  real y[num_data];
}

parameters {
  real c_0;
  real c_1;
  real c_2;
  real c_6_prime;
  real c_7_prime1;
  real c_7_prime2;
  real c_8_prime;
}

transformed parameters {
  real<lower=0> c_6;
  real<lower=-pi(),upper=pi()> c_7;
  real<lower=0> c_8;

  c_6 = exp(c_6_prime);
  c_7 = atan2(c_7_prime1, c_7_prime2);
  c_8 = exp(c_8_prime);

}

model {
  c_0 ~ normal(0.33, 0.1);
  c_1 ~ normal(0.28, 0.1);
  c_2 ~ normal(0.1, 0.1);
  c_6_prime ~ normal(1.16,0.1);
  c_7_prime1 ~ normal(0, 1);
  c_7_prime2 ~ normal(0, 1);
  c_8_prime ~ normal(-4.4,2);
  for(t in 1:num_data) {
    y[t] ~ normal((c_0*(21791*x[t])*(21791*x[t]) + c_1*(21791*x[t]) + c_2 - 300
        - c_6*(sin((2*pi()/365.25)*21791*x[t]-c_7)+0.25*sin(2*(2*pi()/365.25)
        *21791*x[t]-2*c_7)))/150, c_8);
  }
}

generated quantities {
  real y_future[n_future];
  for(t in 1:n_future) {
    y_future[t] = normal_rng((c_0*(21791*(x[t+num_data]))*(21791*(x[t+num_data]))
        + c_1*(21791*(x[t+num_data])) + c_2 - 300 - c_6*(sin((2*pi()/365.25)*21791
        *(x[t+num_data])-c_7)+0.25*sin(2*(2*pi()/365.25)*21791*(x[t+num_data])
        -2*c_7)))/150, c_8);
  }
}
"

In [0]:
# Fit the Stan model.
fit <- stan(
  model_code = model,
  data = data,
  chains = 4,
  warmup = 2000,
  iter = 6000,
  cores = 1,
  refresh = 1000,
  control = list(adapt_delta = 0.999)
)

print(fit, par=c('c_0', 'c_1', 'c_2', 'c_6', 'c_7', 'c_8'), probs=c(.05, .5, 0.95))
samples <- extract(fit)

In [0]:
# Plot of confidence intervals
results <- apply(samples$y_future, 2, quantile, probs=c(0.025, 0.975))

plot(
  x[1:(num_data+n_future)], y[1:(num_data+n_future)],
  col='black', type='l',
  xlim=c(x[1], x[num_data+n_future]),
  ylim=c(min(c(results[1,], y)), max(c(results[2,], y))),
  main='Data, future data, and predicted 95% interval')
lines(x[num_data:(num_data+n_future)], c(y[num_data], results[1,])
        , lty='dashed', col='blue')
lines(x[num_data:(num_data+n_future)], c(y[num_data], results[2,])
        , lty='dashed', col='blue')
abline(v=n, col='red')

In [0]:
# Plot of sample autocorrelation

acf(samples$c_0, main="Autocorrelation of c_0 samples")
acf(samples$c_1, main="Autocorrelation of c_1 samples")
acf(samples$c_2, main="Autocorrelation of c_2 samples")
acf(samples$c_6, main="Autocorrelation of c_6 samples")
acf(samples$c_7, main="Autocorrelation of c_7 samples")
acf(samples$c_8, main="Autocorrelation of c_8 samples")